In [35]:
import gensim
print(dir(gensim))

['NullHandler', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_matutils', 'corpora', 'interfaces', 'logger', 'logging', 'matutils', 'models', 'parsing', 'scripts', 'similarities', 'summarization', 'topic_coherence', 'utils']


In [62]:
import os
cwd = os.getcwd()
sourcefile = cwd + "/sources/pap3.txt"

In [63]:
raw_documents = []

with open(sourcefile) as f: 
    contents = f.read()
    for note in contents.split("#"):
            if len(note) > 0:
                raw_documents.append(note)

In [64]:
print("Number of documents:",len(raw_documents))

Number of documents: 3


In [65]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
# print(gen_docs[0])

In [66]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print("Number of words in dictionary:", len(dictionary))
# for i in dictionary:
#     print(i, dictionary[i])

Number of words in dictionary: 27


In [67]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
# print(corpus)

[[(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(0, 2), (4, 2), (6, 1), (8, 1), (9, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2)], [(0, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]]


In [71]:
tf_idf = gensim.models.TfidfModel(corpus)
# print(tf_idf)
# s = 0
# for i in corpus:
#     print("nowe:",i)
#     s += len(i)
# print(s)

TfidfModel(num_docs=3, num_nnz=33)
nowe: [(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]
nowe: [(0, 2), (4, 2), (6, 1), (8, 1), (9, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2)]
nowe: [(0, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]
33


In [72]:
def get_sim_tfidf(doc):
    query_doc = [w.lower() for w in doc]
#     print(query_doc)
    query_doc_bow = dictionary.doc2bow(query_doc)
#     print(query_doc_bow)
    query_doc_tf_idf = tf_idf[query_doc_bow]
#     print(query_doc_tf_idf)

    sim = gensim.similarities.MatrixSimilarity(corpus) 
    return sim[query_doc_tf_idf]

In [73]:
get_sim_tfidf(gen_docs[0])

array([0.6939868 , 0.18515123, 0.        ], dtype=float32)

In [79]:
import numpy as np

def get_top_n_tfidf(n, doc_id):
    doc_index = doc_id - 1
    top_n = []
    sim = get_sim_tfidf(gen_docs[doc_index])
    top_per_doc = sim.argsort()[-n:][::-1]
    for i in range(0, len(top_per_doc)):
        top_per_doc[i] = top_per_doc[i] + 1
    return top_per_doc

In [81]:
get_top_n_tfidf(3,1)

array([1, 2, 3])

In [47]:
def get_zeros():
    zeros = []
    size = len(dictionary)
    for i in range (0, size):
        row = []
        for r in range (0, size):
            row.append(0)
        zeros.append(row)
    return zeros

# base_matrix = get_zeros()
# for row in base_matrix:
#     print(row)

In [48]:
def get_doc_matrix(doc, k):
    doc_matrix = get_zeros()
    
    query_doc = [w.lower() for w in doc]    
    dict_size = len(dictionary)
    doc_size = len(query_doc)
    
    for i in range(0, doc_size):
        for slide in range(0, k+1):
            if i+slide < doc_size:
                first = dictionary.token2id[query_doc[i]]
                second = dictionary.token2id[query_doc[i+slide]]
                doc_matrix[first][second] += 1
    return doc_matrix
    
# doc_matrix = get_doc_matrix(gen_docs[0], 1)
# for row in doc_matrix:
#     print(row)

In [49]:
def get_doc_vector(doc, k):
    doc_matrix = get_doc_matrix(doc, k)
    doc_vector = []
    
    counter = 0
    
    dict_size = len(dictionary)
    
    for i in range(0, dict_size):
        for j in range(0, dict_size):
            if doc_matrix[i][j] > 0:
                doc_vector.append((counter, doc_matrix[i][j]))
            counter += 1
    return doc_vector

# doc_vector = get_doc_vector(gen_docs[0], 1)
# for el in doc_vector:
#     print(el)

In [56]:
v1 = get_doc_vector(gen_docs[0], 2)
v2 = get_doc_vector(gen_docs[1], 2)
v3 = get_doc_vector(gen_docs[2], 2)

corpus = []
corpus.append(v1)
corpus.append(v2)
corpus.append(v3)

sim = gensim.similarities.MatrixSimilarity(corpus) 
sim[v1]

array([1.        , 0.26179168, 0.11065666], dtype=float32)

In [84]:
def get_graph_corpus(k):
    doc_count = len(gen_docs)
    corpus = []
    for i in range(0, doc_count):
        v = get_doc_vector(gen_docs[i], k)
        corpus.append(v)
    return corpus

In [88]:
def get_sim_graph(doc, k):
    corpus = get_graph_corpus(k)
    doc_vector = get_doc_vector(doc, k)
    sim = gensim.similarities.MatrixSimilarity(corpus) 
    return sim[doc_vector]

In [89]:
def get_top_n_graph(n, doc_id, k):
    doc_index = doc_id - 1
    top_n = []
    sim = get_sim_graph(gen_docs[doc_index], k)
    top_per_doc = sim.argsort()[-n:][::-1]
    for i in range(0, len(top_per_doc)):
        top_per_doc[i] = top_per_doc[i] + 1
    return top_per_doc

In [91]:
get_top_n_graph(3,2,2)

array([2, 1, 3])

In [101]:
def experiment(note_id, k):
    print(get_top_n_tfidf(10,note_id))
    print(get_top_n_graph(10,note_id,k))
    

In [104]:
experiment(2,1)

[2 1 3]
[2 1 3]
